In [6]:
from langchain_groq import ChatGroq

In [7]:
llm = ChatGroq(
    temperature=0, 
    groq_api_key='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', 
    model_name="llama-3.1-70b-versatile"
)

response = llm.invoke("The first person to made aeroplane")
print(response.content)

The invention of the first powered, heavier-than-air aircraft is credited to the Wright brothers, Orville and Wilbur Wright. They designed, built, and flew the first successful airplane on December 17, 1903.

However, there were other inventors and aviation pioneers who made significant contributions to the development of flight before the Wright brothers. Some notable examples include:

1. Sir George Cayley (1773-1857): An English engineer and inventor, Cayley is often credited with designing the first successful glider in 1853. He also identified the principles of lift, drag, and thrust that are still used in aircraft design today.
2. Otto Lilienthal (1848-1896): A German engineer and aviation pioneer, Lilienthal made significant contributions to the development of gliders and was a major influence on the Wright brothers.
3. Octave Chanute (1832-1910): A French-American engineer and aviation pioneer, Chanute was a prominent figure in the development of aviation in the late 19th and e

In [13]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://www.uber.com/global/en/careers/list/133792/")
page_data = loader.load().pop().page_content
print(page_data)

Data Engineer, FinTech - Hyderabad, India | Uber Careers 



        

Please enable JavascriptPlease enable JavascriptSkip to main content

Uber CareersJob search,Data Engineer, FinTechData, Engineering in Hyderabad, IndiaApply NowAbout the RoleWe are seeking a strong and passionate data engineer with experience in large scale system implementation, with a focus on complex data pipelines. The candidate must be able to design and drive large projects from inception to production. The right person will work with stakeholders, analysts, scientists to gather requirements and translate them into a data engineering roadmap. Must be a great communicator, team player, and a technical powerhouse.---- What the Candidate Will Do ----Collaborate with engineering/product/analyst teams across tech sites to collectively accomplish OKRs to take Uber forward.Enrich data layers to effectively deal with the next generation of products which are a result of Uber's big bold bets.Design and build data pipe

In [15]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
print(res.content)

```json
{
  "role": "Data Engineer, FinTech",
  "experience": "Not specified",
  "skills": [
    "Strong SQL skills",
    "Data Warehousing and Data Modelling concepts",
    "Hadoop tech stack: HDFS, Hive, Oozie, Airflow, MapReduce, Spark",
    "Programming languages: Python, Java, Scala",
    "Experience in building ETL Data Pipelines",
    "Performance Troubleshooting and Tuning"
  ],
  "description": "We are seeking a strong and passionate data engineer with experience in large scale system implementation, with a focus on complex data pipelines. The candidate must be able to design and drive large projects from inception to production. The right person will work with stakeholders, analysts, scientists to gather requirements and translate them into a data engineering roadmap. Must be a great communicator, team player, and a technical powerhouse."
}
```


In [16]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Data Engineer, FinTech',
 'experience': 'Not specified',
 'skills': ['Strong SQL skills',
  'Data Warehousing and Data Modelling concepts',
  'Hadoop tech stack: HDFS, Hive, Oozie, Airflow, MapReduce, Spark',
  'Programming languages: Python, Java, Scala',
  'Experience in building ETL Data Pipelines',
  'Performance Troubleshooting and Tuning'],
 'description': 'We are seeking a strong and passionate data engineer with experience in large scale system implementation, with a focus on complex data pipelines. The candidate must be able to design and drive large projects from inception to production. The right person will work with stakeholders, analysts, scientists to gather requirements and translate them into a data engineering roadmap. Must be a great communicator, team player, and a technical powerhouse.'}

In [17]:
type(json_res)

dict

In [19]:
import pandas as pd
df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [20]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [22]:
links = collection.query(query_texts=["Experience in Angular","Expertise in React Native"], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/angular-portfolio'},
  {'links': 'https://example.com/typescript-frontend-portfolio'}],
 [{'links': 'https://example.com/react-native-portfolio'},
  {'links': 'https://example.com/react-portfolio'}]]

In [23]:
job = json_res
job['skills']

['Strong SQL skills',
 'Data Warehousing and Data Modelling concepts',
 'Hadoop tech stack: HDFS, Hive, Oozie, Airflow, MapReduce, Spark',
 'Programming languages: Python, Java, Scala',
 'Experience in building ETL Data Pipelines',
 'Performance Troubleshooting and Tuning']

In [24]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/flutter-portfolio'},
  {'links': 'https://example.com/kotlin-android-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}]]

In [25]:
prompt_email = PromptTemplate.from_template(
    """
    ### JOB DESCRIPTION:
    {job_description}
    
    ### INSTRUCTION:
    You are Mohamed Thanish, a Software Engineer at TTT. TTT is an AI & Software Consulting company dedicated to facilitating
    the seamless integration of business processes through automated tools.
    Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability,
    process optimization, cost reduction, and heightened overall efficiency.
    Your job is to write a cold email to the client regarding the job mentioned above, describing the capability of TTT 
    in fulfilling their needs.
    Also add the most relevant ones from the following links to showcase TTT's portfolio: {link_list}
    Remember you are Mohamed Thanish, Software Engineer at TTT.
    Do not provide a preamble.
    ### EMAIL (NO PREAMBLE):
    
    """
)

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)


Subject: Expert Data Engineering Solutions for Your FinTech Needs

Dear Hiring Manager,

I came across your job posting for a Data Engineer, FinTech, and I'm excited to introduce you to TTT, an AI & Software Consulting company that can help you achieve your data engineering goals. With our expertise in designing and implementing large-scale data pipelines, we're confident that we can support your organization in driving complex data projects from inception to production.

Our team of skilled engineers has extensive experience in working with Hadoop tech stack, including HDFS, Hive, Oozie, Airflow, MapReduce, and Spark. We're also proficient in programming languages such as Python, Java, and Scala, which aligns with your requirements. Our expertise in building ETL Data Pipelines and Performance Troubleshooting and Tuning will ensure that your data engineering roadmap is executed efficiently.

At TTT, we've empowered numerous enterprises with tailored solutions that foster scalability, p

In [26]:
import os
os.getcwd()


'C:\\Users\\Moham\\cold_email_generation_tool'